In [1]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

import os

import time

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
df = pd.read_csv('data/cities.csv', encoding='latin-1')


In [4]:
df.count()

Country      193
Latitude     193
Longitude    193
Capital      193
elevation    193
dis          193
dtype: int64

In [ ]:
timezones_df = pd.read_json('data/countries.json')
timezones_df.head()

In [ ]:
timezones_df.count()

In [ ]:
# only get name and timezone
timezones_df = timezones_df[['name', 'timezones']]
# map timezones to first element of list

new_df = pd.DataFrame(columns=['name', 'timezones'])

for index, row in timezones_df.iterrows():
    if row['timezones'] is not None:
        new_row = pd.DataFrame({'name': [row['name']], 'timezones': [row['timezones'][0]["zoneName"]]})
        new_df = pd.concat([new_df, new_row], ignore_index=True)

new_df.head()

In [ ]:
new_df.count()

In [ ]:
countries_timezones = pd.merge(df, new_df, how='left', left_on='Country', right_on='name')
countries_timezones.head()

In [ ]:
countries_timezones.count()

In [ ]:
countries_timezones = countries_timezones[['Country', 'Capital', 'Latitude', 'Longitude', 'timezones']]
countries_timezones.head()
countries_timezones.to_csv('data/countries_timezones.csv', index=False)

In [9]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

for index, row in df.iterrows():
	if not (os.path.exists(f'output/{row["Capital"]}_hourly.csv'.replace("*", ""))):
		url = "https://archive-api.open-meteo.com/v1/archive"
		params = {
			"latitude": row["Latitude"],
			"longitude": row["Longitude"],
			"start_date": "2023-01-01",
			"end_date": "2023-12-31",
			"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
			"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
			}
		responses = openmeteo.weather_api(url, params=params)

		# Process first location. Add a for-loop for multiple locations or weather models
		response = responses[0]
		print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
		print(f"Elevation {response.Elevation()} m asl")
		print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
		print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

		# Process hourly data. The order of variables needs to be the same as requested.
		hourly = response.Hourly()
		hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
		hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
		hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
		hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
		hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
		hourly_rain = hourly.Variables(5).ValuesAsNumpy()
		hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
		hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
		hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
		hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
		hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
		hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
		hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
		hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
		hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
		hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
		hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
		hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
		hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
		hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
		hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
		hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
		hourly_soil_temperature_0_to_7cm = hourly.Variables(22).ValuesAsNumpy()
		hourly_soil_temperature_7_to_28cm = hourly.Variables(23).ValuesAsNumpy()
		hourly_soil_temperature_28_to_100cm = hourly.Variables(24).ValuesAsNumpy()
		hourly_soil_temperature_100_to_255cm = hourly.Variables(25).ValuesAsNumpy()
		hourly_soil_moisture_0_to_7cm = hourly.Variables(26).ValuesAsNumpy()
		hourly_soil_moisture_7_to_28cm = hourly.Variables(27).ValuesAsNumpy()
		hourly_soil_moisture_28_to_100cm = hourly.Variables(28).ValuesAsNumpy()
		hourly_soil_moisture_100_to_255cm = hourly.Variables(29).ValuesAsNumpy()

		hourly_data = {"date": pd.date_range(
			start = pd.to_datetime(hourly.Time(), unit = "s"),
			end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
			freq = pd.Timedelta(seconds = hourly.Interval()),
			inclusive = "left"
		)}
		hourly_data["temperature_2m"] = hourly_temperature_2m
		hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
		hourly_data["dew_point_2m"] = hourly_dew_point_2m
		hourly_data["apparent_temperature"] = hourly_apparent_temperature
		hourly_data["precipitation"] = hourly_precipitation
		hourly_data["rain"] = hourly_rain
		hourly_data["snowfall"] = hourly_snowfall
		hourly_data["snow_depth"] = hourly_snow_depth
		hourly_data["weather_code"] = hourly_weather_code
		hourly_data["pressure_msl"] = hourly_pressure_msl
		hourly_data["surface_pressure"] = hourly_surface_pressure
		hourly_data["cloud_cover"] = hourly_cloud_cover
		hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
		hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
		hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
		hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
		hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
		hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
		hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
		hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
		hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
		hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
		hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
		hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
		hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
		hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
		hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
		hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
		hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
		hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm

		hourly_dataframe = pd.DataFrame(data = hourly_data)
		hourly_dataframe.to_csv(f'output/{row["Capital"]}_hourly.csv'.replace("*", ""))


Coordinates 3.1282951831817627°E 101.68547821044922°N
Elevation 47.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 4.182776927947998°E 73.54285430908203°N
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 12.618629455566406°E -7.981903076171875°N
Elevation 343.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 35.88751983642578°E 14.534884452819824°N
Elevation 44.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 7.06502628326416°E 171.4104766845703°N
Elevation 4.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 18.101932525634766°E -15.949371337890625°N
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates -20.14059829711914°E 57.53506851196289°N
Elevation 4.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 19.43760871887207°E -99.10714721679688°N
Elevation 2234.0 m asl
Timezone None None
Timezone difference to GM

In [ ]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesAsNumpy()
daily_sunset = daily.Variables(8).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(19).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)

print(daily_dataframe)

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 31.769,
	"longitude": 35.2163,
	"start_date": "2023-01-01",
	"end_date": "2023-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
    }
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(22).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(23).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(24).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm = hourly.Variables(25).ValuesAsNumpy()
hourly_soil_moisture_0_to_7cm = hourly.Variables(26).ValuesAsNumpy()
hourly_soil_moisture_7_to_28cm = hourly.Variables(27).ValuesAsNumpy()
hourly_soil_moisture_28_to_100cm = hourly.Variables(28).ValuesAsNumpy()
hourly_soil_moisture_100_to_255cm = hourly.Variables(29).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm
hourly_data["soil_moisture_0_to_7cm"] = hourly_soil_moisture_0_to_7cm
hourly_data["soil_moisture_7_to_28cm"] = hourly_soil_moisture_7_to_28cm
hourly_data["soil_moisture_28_to_100cm"] = hourly_soil_moisture_28_to_100cm
hourly_data["soil_moisture_100_to_255cm"] = hourly_soil_moisture_100_to_255cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe.to_csv("Jerusalem_hourly.csv")
print(hourly_dataframe)


Coordinates 31.739892959594727°E 35.22208786010742°N
Elevation 792.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                    date  temperature_2m  relative_humidity_2m  dew_point_2m  \
0    2023-01-01 00:00:00        8.427501             85.449944        6.1275   
1    2023-01-01 01:00:00        8.027500             87.498283        6.0775   
2    2023-01-01 02:00:00        7.677500             89.298538        6.0275   
3    2023-01-01 03:00:00        7.477500             90.837929        6.0775   
4    2023-01-01 04:00:00        7.327500             92.090500        6.1275   
...                  ...             ...                   ...           ...   
8755 2023-12-31 19:00:00        9.177501             87.605087        7.2275   
8756 2023-12-31 20:00:00       11.327500             78.257767        7.6775   
8757 2023-12-31 21:00:00       10.127501             79.974556        6.8275   
8758 2023-12-31 22:00:00        9.127501             81.222641        6.0